<a href="https://colab.research.google.com/github/UdeS-PMF/gmq710_test/blob/develop/Pratique_OpenCV_avec_la_vid%C3%A9o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse de séquence vidéo avec OpenCV

## Objectif

On souhaite lire un flux vidéo et mettre en place une détection automatique des objets. Il s'agit de la même approche que pour l'analyse d'image. Un flux vidéo est une séquence d'images (frame). Chaque frame sera analysée.

## Principe

On lit un fichier MP4 (on pourrait analyser la sortie d'une caméra). On utilise OpenCV pour lire la vidéo et décomposer en images (frame by frame).

On peut ainsi analyser chaque frame (comme une image) et on applique une détection par réseaux de neurones.

On utilise un réseau léger (mobilenet), les résultats sont moins bons qu'un réseau plus complexe, toutefois l'approche est peu coûteuse en ressources, ce qui permet de l'utiliser sur un appareil embarqué.

On peut télécharger le réseau à cette adresse : https://drive.google.com/file/d/1VaJwWPOQVNzbS6PfuS05oZdlghA8ovio/view?usp=sharing

In [ ]:
!unzip /content/mobilenet.zip -d /content/ssd_mobilenet

Archive:  /content/mobilenet.zip
  inflating: /content/ssd_mobilenet/frozen_inference_graph.pb  
  inflating: /content/ssd_mobilenet/labels.txt  
  inflating: /content/ssd_mobilenet/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt  


## Importation

On importe numpy, opencv

In [ ]:
import numpy as np
import cv2
import datetime
import os

On lit un fichier mp4.

In [ ]:
# on lit un fichier mp4
filename = '/content/3727445-hd_1920_1080_30fps.mp4'
video_cap = cv2.VideoCapture(filename)
# on récupère la taille des frames
frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_cap.get(cv2.CAP_PROP_FPS))
print(frame_height, frame_width, fps)

1080 1920 29


On forme le réseau

In [ ]:
weights = "/content/ssd_mobilenet/frozen_inference_graph.pb"
model = "/content/ssd_mobilenet/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
net = cv2.dnn.readNetFromTensorflow(weights, model)

On définit les classes recherchées

In [ ]:
class_names = []
with open("/content/ssd_mobilenet/labels.txt", "r") as f:
    class_names = f.read().strip().split("\n")

On va essayer de détecter les camions dans la vidéo. L'idée est de faire analyser les frames par un réseau de neurones et de conserver les images avec un camion.

In [ ]:
# on définit des couleurs aléatoires
colors = np.random.randint(0, 255, size=(len(class_names), 3))

# on va créer un répertoire pour les images avec les camions
if not os.path.exists('/content/results'):
    # Créez le répertoire
    os.makedirs('/content/results')

# liste des frames avec un camion
truck_detected = []
# on lit la vidéo
while video_cap.isOpened():

    # on ouvre la vidéo
    success, frame = video_cap.read()
    # si l'ouverture est faite
    if success == True:
        # on récupère la taille de l'image
        h = frame.shape[0]
        w = frame.shape[1]
        # on va créer une image à analyser (320 x 320)
        blob = cv2.dnn.blobFromImage(
            frame, 1.0 / 127.5, (320, 320), [127.5, 127.5, 127.5], swapRB=True)
        # on envoie au réseau
        net.setInput(blob)
        output = net.forward()
        # on définit un FLAG pour savoir si il y a un camion
        truck = False
        # on souhaite récupérer les camions si on est sûr à au moins 65%
        max_detect = 0.65
        for detection in output[0, 0, :, :]:

            # on analyse la détection du réseau
            # on récupère la probabilité de détection d'un objet (quelconque)
            probability = detection[2]
            if probability > max_detect:
                # on note la classe identifiée
                class_id = int(detection[1])
                label = class_names[class_id - 1].upper()
                color = colors[class_id]
                B, G, R = int(color[0]), int(color[1]), int(color[2])
                # on récupère la zone dans la vidéo
                box = [int(a * b) for a, b in zip(detection[3:7], [w, h, w, h])]
                box = tuple(box)
                # si c'est un camion on va conserver les infos
                if label == "TRUCK":
                    # on note une info au dessus de la boite
                    text = f"{label} {probability * 100:.2f}%"
                    cv2.putText(frame, text, (box[0], box[1]),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    cv2.rectangle(frame, box[:2], box[2:], (B, G, R), thickness=2)

                    # on récupère le temps de la frame
                    millis = video_cap.get(cv2.CAP_PROP_POS_MSEC)
                    # on va déterminer les heures, minutes, secondes
                    seconds, millis = divmod(millis, 1000)
                    minutes, seconds = divmod(seconds, 60)
                    hours, minutes = divmod(minutes, 60)
                    days, hours = divmod(hours, 24)
                    seconds = seconds + millis / 1000
                    # on peut sauvegarder l'image avec l'infos de temps
                    outfilename = f'/content/results/image_resu_{int(hours)}_{int(minutes)}_{round(seconds,3)}_p({round(probability*100,2)}%).jpg'
                    # on ajoute la frame et le nom du fichier
                    truck_detected.append([outfilename, frame])


        # on stoppe tout si il y a un pb
    else:
        break

# on doit relâcher la vidéo à la fin
video_cap.release()


On peut générer les images de chaque frame

In [ ]:
for detect in truck_detected:
  cv2.imwrite(detect[0], detect[1])

On peut effacer les résultats

In [ ]:
for file in os.listdir('/content/results/'):
  if 'image_resu' in file:
    os.remove('/content/results/' + file)
